# Constraining subglacial processes from surface velocity observations using surrogate-based Bayesian inference
## Part 1 - Training an ensemble of neural networks

In this notebook, we will illustrate the process of using Bayesian Bootstrap Aggregation (BayesBag) to train an ensemble of neural networks.  In this case, each ensemble member is one possible surrogate for the coupled hydrology-ice dynamics model described in the paper, mapping from a vector of 8 parameters to a velocity field.  We begin by importing both the parameters and the associated velocity fields computed by the physics model, which will act as training data for the surrogate.

In [ ]:
import pickle
import numpy as np
import utilities

# Load velocity fields
F_lin = pickle.load(open('data/F_prior.p','rb'))

# Load model parameters 
X = pickle.load(open('data/X_prior.p','rb'))

The velocity fields have some bad simulations in them, so we filter out circumstances in which the model never ran past 12 years, and in which the max velocity was greater than 100km/a.

In [ ]:
p = (X[:,1]<1e5)*(X[:,3]>=12)

F_lin = F_lin[p]
X = X[p,6:]

Finally, we log transform the velocity fields.

In [ ]:
F = np.log10(F_lin)

We will use pytorch to construct and train the neural networks.  To this end, we will move the physical model's parameters and (log-)speed fields to pytorch, and use the GPU if it's available.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

m = X.shape[0]

X = torch.from_numpy(X)
F = torch.from_numpy(F)

X = X.to(torch.float32)
F = F.to(torch.float32)

X = X.to(device)
F = F.to(device)

X_hat = torch.log10(X)

Part of our objective function is to weight by element area.  We will grab those values from a .vtu of an observed velocity field.

In [ ]:
u_obs = utilities.VData('./data/u_observed.vtu')
point_area = torch.tensor(u_obs.get_point_area(),dtype=torch.float,device=device)
normed_area = point_area/point_area.sum()

Next we need to define a few functions and classes.  First, we will create a function that extracts eigenglaciers and constructs the matrix $\hat{V}$, corresponding to the Dimensionality Reduction section.

In [ ]:
def get_eigenglaciers(omegas,F,cutoff=0.999):
    F_mean = (F*omegas).sum(axis=0)
    F_bar = F - F_mean # Eq. 28
    S = F_bar.T @ torch.diag(omegas.squeeze()) @ F_bar # Eq. 27
    lamda, V = torch.eig(S,eigenvectors=True) # Eq. 26
    lamda = lamda[:,0].squeeze()
    
    cutoff_index = torch.sum(torch.cumsum(lamda/lamda.sum(),0)<cutoff)
    lamda_truncated = lamda.detach()[:cutoff_index]
    V = V.detach()[:,:cutoff_index]
    V_hat = V @ torch.diag(torch.sqrt(lamda_truncated)) # A slight departure from the paper: Vhat is the
                                                        # eigenvectors scaled by the eigenvalue size.  This
                                                        # has the effect of allowing the outputs of the neural
                                                        # network to be O(1).  Otherwise, it doesn't make 
                                                        # any difference.
    return V_hat, F_bar, F_mean

Second, we define the architecture of the neural network to be used as a surrogate.  This corresponds to the architecture defined in Fig. 3.

In [ ]:
import torch.nn as nn

class Emulator(nn.Module):
    def __init__(self,n_parameters,n_eigenglaciers,n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,V_hat,F_mean):
        super().__init__()
        # Inputs to hidden layer linear transformation
        self.l_1 = nn.Linear(n_parameters, n_hidden_1)
        self.norm_1 = nn.LayerNorm(n_hidden_1)
        self.dropout_1 = nn.Dropout(p=0.0)
        self.l_2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.norm_2 = nn.LayerNorm(n_hidden_2)
        self.dropout_2 = nn.Dropout(p=0.5)
        self.l_3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.norm_3 = nn.LayerNorm(n_hidden_3)
        self.dropout_3 = nn.Dropout(p=0.5)
        self.l_4 = nn.Linear(n_hidden_3, n_hidden_4)
        self.norm_4 = nn.LayerNorm(n_hidden_3)
        self.dropout_4 = nn.Dropout(p=0.5)
        self.l_5 = nn.Linear(n_hidden_4, n_eigenglaciers)

        self.V_hat = torch.nn.Parameter(V_hat,requires_grad=False)
        self.F_mean = torch.nn.Parameter(F_mean,requires_grad=False)

    def forward(self, x, add_mean=False):
        # Pass the input tensor through each of our operations

        a_1 = self.l_1(x)
        a_1 = self.norm_1(a_1)
        a_1 = self.dropout_1(a_1)
        z_1 = torch.relu(a_1) 

        a_2 = self.l_2(z_1)
        a_2 = self.norm_2(a_2)
        a_2 = self.dropout_2(a_2)
        z_2 = torch.relu(a_2) + z_1

        a_3 = self.l_3(z_2)
        a_3 = self.norm_3(a_3)
        a_3 = self.dropout_3(a_3)
        z_3 = torch.relu(a_3) + z_2
        
        a_4 = self.l_4(z_3)
        a_4 = self.norm_3(a_4)
        a_4 = self.dropout_3(a_4)
        z_4 = torch.relu(a_4) + z_3
        
        z_5 = self.l_5(z_4)
        if add_mean:
            F_pred = z_5 @ self.V_hat.T + self.F_mean
        else:
            F_pred = z_5 @ self.V_hat.T

        return F_pred 

Third, we create an optimization procedure that trains a model for a given set of instance weights ($\omega_d$) and training data.  Optimization is performed using mini-batch gradient descent.

In [ ]:
from torch.utils.data import TensorDataset

def criterion_ae(F_pred,F_obs,omegas,area):
    instance_misfit = torch.sum(torch.abs((F_pred - F_obs))**2*area,axis=1)
    return torch.sum(instance_misfit*omegas.squeeze())

def train_surrogate(e,X_train,F_train,omegas,area,batch_size=128,epochs=3000,eta_0=0.01,k=1000.):
    
    omegas_0 = torch.ones_like(omegas)/len(omegas)
    training_data = TensorDataset(X_train,F_train,omegas)

    batch_size = 128
    train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                               batch_size=batch_size, 
                                               shuffle=True)
    
    optimizer = torch.optim.Adam(e.parameters(),lr=eta_0,weight_decay=0.0)
    
    # Loop over the data
    for epoch in range(epochs):
        # Loop over each subset of data
        for param_group in optimizer.param_groups:
            param_group['lr'] = eta_0*(10**(-epoch/k))

        for x,f,o in train_loader:
            e.train()
            # Zero out the optimizer's gradient buffer
            optimizer.zero_grad()

            f_pred = e(x)

            # Compute the loss
            loss = criterion_ae(f_pred,f,o,area)

            # Use backpropagation to compute the derivative of the loss with respect to the parameters
            loss.backward()
        
            # Use the derivative information to update the parameters
            optimizer.step()
            
        e.eval()
        F_train_pred = e(X_train)
        # Make a prediction based on the model
        loss_train = criterion_ae(F_train_pred,F_train,omegas,area)
        # Make a prediction based on the model
        loss_test = criterion_ae(F_train_pred,F_train,omegas_0,area)

        # Print the epoch, the training loss, and the test set accuracy.
        if epoch%10==0:
            print(epoch,loss_train.item(),loss_test.item())

    

Here we put it all together: loop over the desired number of models, drawing random Bayesian bootstrap weights for each, training the surrogate, and saving the resulting models.  

In [ ]:
from scipy.stats import dirichlet

torch.manual_seed(0)
np.random.seed(0)

n_parameters = X_hat.shape[1]
n_hidden_1 = 128
n_hidden_2 = 128
n_hidden_3 = 128
n_hidden_4 = 128

n_models = 3 #To reproduce the paper, this should be 50
for model_index in range(n_models):
    omegas = torch.tensor(dirichlet.rvs(np.ones(m)),dtype=torch.float,device=device).T

    V_hat, F_bar, F_mean = get_eigenglaciers(omegas,F)
    n_eigenglaciers = V_hat.shape[1]

    e = Emulator(n_parameters,n_eigenglaciers,n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,V_hat,F_mean)
    e.to(device)

    train_surrogate(e,X_hat,F_bar,omegas,normed_area,epochs=3000)
    torch.save(e.state_dict(),'emulator_ensemble/emulator_{0:03d}.h5'.format(model_index))


## Part 2 - MCMC over the ensemble
Now that a number of neural network surrogates have been trained on random subsets of high-fidelity model runs, we will perform Markov Chain Monte Carlo sampling over each of these surrogates.  The correct parameter distribution for the high-fidelity model will be approximated by concatenating the Markov Chains over all of the surrogates.

In [ ]:
import pickle
import numpy as np
import utilities

Read in the models trained above.

In [ ]:
models = []
n_models = 3 #To reproduce the paper, this should be 50

for i in range(n_models):
    state_dict = torch.load('emulator_ensemble/emulator_{0:03d}.h5'.format(i))
    e = Emulator(state_dict['l_1.weight'].shape[1],state_dict['V_hat'].shape[1],n_hidden_1,n_hidden_2,n_hidden_3,n_hidden_4,state_dict['V_hat'],state_dict['F_mean'])
    e.load_state_dict(state_dict)
    e.to(device)
    e.eval()
    models.append(e)

Read in some relevant training data and ancillary values.  Convert observed velocities to speeds.

In [ ]:
u_obs = utilities.VData('./data/u_observed.vtu')
v_obs = utilities.VData('./data/v_observed.vtu')
H_obs = utilities.VData('./data/H_observed.vtu')

H = torch.tensor(H_obs.u)
H = H.to(torch.float32).to(device)

U_obs = torch.tensor(((np.sqrt(u_obs.u**2 + v_obs.u**2))))
U_obs = U_obs.to(torch.float32).to(device)

Define the likelihood model, which requires a parameterization of observational uncertainty.

In [ ]:
from scipy.spatial.distance import pdist, squareform
D = torch.tensor(squareform(pdist(u_obs.x)),dtype=torch.float32,device=device)

sigma2 = 10**2
sigma_flow2 = 10**2
alpha_cov = 1

l_model = 4*torch.sqrt(H.unsqueeze(1) @ H.unsqueeze(0))
Sigma_obs = sigma2*torch.eye(D.shape[0],device=device)
Sigma_flow = sigma_flow2*(1 + D**2/(2*alpha_cov*l_model**2))**-alpha_cov
Sigma = Sigma_obs + Sigma_flow

Construct the precision matrix (the inverse of equation 50)

In [ ]:
rho = 1./(1e4**2)
K = torch.diag(point_area*rho)
Tau = K @ torch.inverse(Sigma) @ K

Construct the Beta prior distribution.  

In [ ]:
from scipy.stats import beta
alpha_b = 3.0
beta_b = 3.0

X_min = X_hat.cpu().numpy().min(axis=0)-1e-3
X_max = X_hat.cpu().numpy().max(axis=0)+1e-3

X_prior = beta.rvs(alpha_b,beta_b,size=(10000,8))*(X_max - X_min) + X_min

X_min = torch.tensor(X_min,dtype=torch.float32,device=device)
X_max = torch.tensor(X_max,dtype=torch.float32,device=device)

This function returns a value that is proportional to the negative log-posterior distribution (The summands of equation 53).  

In [ ]:
def V(X):
    U_pred = 10**m(X,add_mean=True)
    
    r = (U_pred - U_obs)
    X_bar = (X - X_min)/(X_max - X_min)

    L1 = -0.5*r @ Tau @ r
    L2 = torch.sum((alpha_b-1)*torch.log(X_bar) + (beta_b-1)*torch.log(1-X_bar)) 

    return -(L1 + L2)

We use the Metropolis-adjusted Langevin Algorithm to sample from the posterior distribution, which benefits from the availability of gradient and Hessian information.  Here, we compute these quantities (and some helpful additional ones) using automatic differentiation in pytorch.

In [ ]:
def get_log_like_gradient_and_hessian(V,X,eps=1e-2,compute_hessian=False):
    log_pi = V(X)
    if compute_hessian:
        g = torch.autograd.grad(log_pi,X,retain_graph=True,create_graph=True)[0]
        H = torch.stack([torch.autograd.grad(e,X,retain_graph=True)[0] for e in g])
        lamda,Q = torch.eig(H,eigenvectors=True)
        lamda_prime = torch.sqrt(lamda[:,0]**2 + eps)
        lamda_prime_inv = 1./torch.sqrt(lamda[:,0]**2 + eps)
        H = Q @ torch.diag(lamda_prime) @ Q.T
        Hinv = Q @ torch.diag(lamda_prime_inv) @ Q.T
        log_det_Hinv = torch.sum(torch.log(lamda_prime_inv))
        return log_pi,g,H,Hinv,log_det_Hinv
    else: 
        return log_pi

We initialize the sampler by first finding the Maximum A Posteriori parameter value, or MAP point.  We find the MAP point using gradient descent paired with a simple line search.

In [ ]:
def find_MAP(X,n_iters=50,print_interval=10):
    print('***********************************************')
    print('***********************************************')
    print('Finding MAP point')
    print('***********************************************')
    print('***********************************************')
    # Line search distances
    alphas = np.logspace(-4,0,11)
    # Find MAP point
    for i in range(n_iters):
        log_pi,g,H,Hinv,log_det_Hinv = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        p = Hinv @ -g
        alpha_index = np.nanargmin([get_log_like_gradient_and_hessian(V,X + alpha*p,compute_hessian=False).detach().cpu().numpy() for alpha in alphas])
        mu = X + alphas[alpha_index] * p 
        X.data = mu.data
        if i%print_interval==0:
            print('===============================================')
            print('iter: {0:d}, ln(P): {1:6.1f}, curr. m: {2:4.4f},{3:4.2f},{4:4.2f},{5:4.2f},{6:4.2f},{7:4.2f},{8:4.2f},{9:4.2f}'.format(i,log_pi,*X.data.cpu().numpy()))
            print('===============================================')
    return X

With a good initial guess for the sampler discovered, we now implement the MALA algorithm.  

In [ ]:
def draw_sample(mu,cov,eps=1e-10):
    L = torch.cholesky(cov + eps*torch.eye(cov.shape[0],device=device))
    return mu + L @ torch.randn(L.shape[0],device=device)

def get_proposal_likelihood(Y,mu,inverse_cov,log_det_cov):
    return -0.5*log_det_cov - 0.5*(Y - mu) @ inverse_cov @ (Y-mu)

def MALA_step(X,h,local_data=None):
    if local_data is not None:
        pass  
    else:
        local_data = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        
    log_pi,g,H,Hinv,log_det_Hinv = local_data
    
    X_ = draw_sample(X,2*h*Hinv).detach()
    X_.requires_grad=True
    
    log_pi_ = get_log_like_gradient_and_hessian(V,X_,compute_hessian=False)

    logq = get_proposal_likelihood(X_,X,H/(2*h),log_det_Hinv)
    logq_ = get_proposal_likelihood(X,X_,H/(2*h),log_det_Hinv)

    log_alpha = (-log_pi_ + logq_ + log_pi - logq)
    alpha = torch.exp(min(log_alpha,torch.tensor([0.],device=device)))
    u = torch.rand(1,device=device)
    if u <= alpha and log_alpha!=np.inf:
        X.data = X_.data
        local_data = get_log_like_gradient_and_hessian(V,X,compute_hessian=True)
        s = 1
    else:
        s = 0
    return X,local_data,s

def MALA(X,n_iters=10001,h=0.1,acc_target=0.25,k=0.01,beta=0.99,sample_path='./samples/',model_index=0,save_interval=1000,print_interval=50):
    print('***********************************************')
    print('***********************************************')
    print('Running Metropolis-Adjusted Langevin Algorithm for model index {0}'.format(model_index))
    print('***********************************************')
    print('***********************************************')
    local_data = None
    vars = []
    acc = acc_target
    for i in range(n_iters):
        X,local_data,s = MALA_step(X,h,local_data=local_data)
        vars.append(X.detach())
        acc = beta*acc + (1-beta)*s
        h = min(h*(1+k*np.sign(acc - acc_target)),h_max)
        if i%print_interval==0:
            print('===============================================')
            print('sample: {0:d}, acc. rate: {1:4.2f}, log(P): {2:6.1f}'.format(i,acc,local_data[0].item()))
            print('curr. m: {0:4.4f},{1:4.2f},{2:4.2f},{3:4.2f},{4:4.2f},{5:4.2f},{6:4.2f},{7:4.2f}'.format(*X.data.cpu().numpy()))
            print('===============================================')
          
        if i%save_interval==0:
            print('///////////////////////////////////////////////')
            print('Saving samples for model {0:03d}'.format(model_index))
            print('///////////////////////////////////////////////')
            X_posterior = torch.stack(vars).cpu().numpy()
            np.save(open(sample_path+'X_posterior_model_{0:03d}.npy'.format(model_index),'wb'),X_posterior)
    X_posterior = torch.stack(vars).cpu().numpy()
    return X_posterior       

We now run the MAP/MALA procedure for each surrogate in the bootstrapped ensemble, and save the resulting posterior distributions.

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
for j,m in enumerate(models):
    X = torch.tensor(X_prior[np.random.randint(X_prior.shape[0],size=5)].mean(axis=0),requires_grad=True,dtype=torch.float,device=device)
    X = find_MAP(X)
    # To reproduce the paper, n_iters should be 10^5
    X_posterior = MALA(X,n_iters=10000,model_index=j,save_interval=1000,print_interval=100)

TODO: Add plotting